In [1]:
import pandas as pd
import findspark
import functools as ft
import nltk
import string
import re

In [2]:
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.stem import WordNetLemmatizer

# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
import findspark
from pyspark import SparkContext

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import Evaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
import os, sys
from pyspark.sql import SparkSession
import pyspark.sql.dataframe

findspark.init()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder.master('local[*]').appName("ml_example").getOrCreate()

sc = spark.sparkContext

In [5]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import NaiveBayesModel

In [6]:

psy_df = pd.read_csv('data\Youtube01-Psy.csv')
katy_df = pd.read_csv('data\Youtube02-KatyPerry.csv')
lmfao_df = pd.read_csv('data\Youtube03-LMFAO.csv')
eminem_df = pd.read_csv('data\Youtube04-Eminem.csv')
shakira_df = pd.read_csv('data\Youtube05-Shakira.csv')

dataframes = [psy_df, katy_df, lmfao_df, eminem_df, shakira_df]

main_df = ft.reduce(lambda left, right: pd.concat([left, right], axis=0, ignore_index=True), dataframes)
main_df

,COMMENT_ID,AUTHOR,DATE,CONTENT,label
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...,...,...,...
1951,_2viQ_Qnc6-bMSjqyL1NKj57ROicCSJV5SwTrw-RFFA,Katie Mettam,2013-07-13T13:27:39.441000,I love this song because we sing it at Camp al...,0
1952,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13T13:14:30.021000,I love this song for two reasons: 1.it is abou...,0
1953,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13T12:09:31.188000,wow,0
1954,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13T11:17:52.308000,Shakira u are so wiredo,0


## Steps to Follow:-
1. Remove punctuation
2. Remove Stopwords
3. Remove emojis
4. Lemmatization
5. Tokenization
6. Scaling
7. Implement Spark

In [7]:
main_df['label'].value_counts()

1    1005
0     951
Name: label, dtype: int64

In [8]:
main_df.drop(['COMMENT_ID', 'AUTHOR', 'DATE'], axis=1, inplace=True)
main_df

,CONTENT,label
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...
1951,I love this song because we sing it at Camp al...,0
1952,I love this song for two reasons: 1.it is abou...,0
1953,wow,0
1954,Shakira u are so wiredo,0


## Adding single letters to the list of stopwords

In [9]:
alphabet_list = list(string.ascii_lowercase)
stop_words = stopwords.words('english')

for i in range(len(alphabet_list)):
    if (alphabet_list[i] in stop_words):
        continue
    else:
        stop_words.append(alphabet_list[i])
        
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Removing Punctuations and Random Characters

In [10]:
def clean_text(df, text):
    df[text] = df[text].str.lower()
    df[text] = df[text].apply(lambda elem: re.sub(r'@[A-Za-z0-9]+', '', elem))              # Removing all @mentions
    df[text] = df[text].apply(lambda elem: re.sub(r'https?:\/\/\S+', '', elem))             # Removing all links
    df[text] = df[text].apply(lambda elem: re.sub(r'#', '', elem))                          # Removing the hashtag symbol
    df[text] = df[text].apply(lambda elem: re.sub(r"([^0-9A-Za-z \t])", "", elem))          # Removing all non-alphanumeric symbols
    df[text] = df[text].apply(lambda elem: re.sub(r"\w*\d\w*", "", elem))                   # Removing all numbers
    df[text] = df[text].apply(lambda elem: re.sub(r"^\d+\w|^\w+\d+\w|^\w+\d$", "", elem))
    
    return df

In [11]:
clean_df = clean_text(main_df, 'CONTENT')
clean_df

,CONTENT,label
0,huh anyway check out this youtube channel,1
1,hey guys check out my new channel and our firs...,1
2,just for test i have to say murdevcom,1
3,me shaking my sexy ass on my channel enjoy,1
4,watchvvtarggvgtwq check this out,1
...,...,...
1951,i love this song because we sing it at camp al...,0
1952,i love this song for two reasons is about afr...,0
1953,wow,0
1954,shakira u are so wiredo,0


## Removing Stopwords

In [12]:
def remove_stopwords(df, text):
    
    alphabet_list = list(string.ascii_lowercase)
    stop_words = stopwords.words('english')

    for i in range(len(alphabet_list)):
        if (alphabet_list[i] in stop_words):
            continue
        else:
            stop_words.append(alphabet_list[i])
        
    ABC = list((df[text]))
    clean_lst = []
    for i in range(ABC.__len__()):
        result_words = [word for word in ABC[i].split(' ') if word not in stop_words]
        str_result = ' '.join(result_words)
        clean_lst.append(str_result)
        
    cleaner_df = pd.DataFrame(clean_lst, columns=['CONTENT'])
    
    return cleaner_df

cleaner_df = remove_stopwords(clean_df, 'CONTENT')
cleaner_df.head(60)

cleaner_df.dropna(inplace=True)

## Removing Emojis

In [13]:
cleaner_df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
# cleaner_df.to_csv('data/temp_data.csv',index=False)

,CONTENT
0,huh anyway check youtube channel
1,hey guys check new channel first vid us monke...
2,test say murdevcom
3,shaking sexy ass channel enjoy
4,watchvvtarggvgtwq check
...,...
1951,love song sing camp time
1952,love song two reasons africa born beautiful ...
1953,wow
1954,shakira wiredo


## Lemmatization

In [14]:
lemmatizer = WordNetLemmatizer()



""" def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'V': wordnet.VERB,
        'N': wordnet.NOUN,
        'J': wordnet.ADJ,
        'R': wordnet.ADV
    }
    
    return tag_dict.get(tag) """

def lem_df(df, text):
    lst_sent = list(df[text])
    lem_lst = []
    for i in range(len(lst_sent)):
        input = nltk.word_tokenize(lst_sent[i])
        result_words = [lemmatizer.lemmatize(word) for word in input]
        str_result = ' '.join(result_words)
        lem_lst.append(str_result)
        
    cleanest_df = pd.DataFrame(lem_lst, columns=['CONTENT'])
    return cleanest_df

lemmatized_df = lem_df(cleaner_df, 'CONTENT')
lemmatized_df = lemmatized_df.join(clean_df['label'])
lemmatized_df.dropna(inplace=True)
lemmatized_df.to_csv('data/full_comment_data.csv', index=False)

## TFIDF Vectorization

In [24]:
docs = spark.read.csv('data/comment_data.csv', header=True, inferSchema=True)

docs_rdd = docs.rdd
tokenize = Tokenizer(inputCol='CONTENT', outputCol='WORDS')
words_data = tokenize.transform(docs.dropna())

hashingTF = HashingTF(inputCol="WORDS", outputCol="tf_features", numFeatures=20)
featurized_data = hashingTF.transform(words_data)

idf = IDF(inputCol="tf_features", outputCol="features")
idfModel = idf.fit(featurized_data)
rescaledData = idfModel.transform(featurized_data)

# rescaledData.show()
final_data = rescaledData.drop('CONTENT', 'WORDS', 'tf_features')
final_data.show()

splits = final_data.randomSplit([0.8, 0.2], seed=69)
train_data = splits[0]
test_data = splits[1]
test_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|(20,[1,9,12,17],[...|
|    1|(20,[1,2,3,5,6,7,...|
|    1|(20,[2,6,8],[1.30...|
|    1|(20,[5,7,9,12,17]...|
|    1|(20,[1,8],[1.0256...|
|    1|(20,[1,2,3,5,8,11...|
|    1|(20,[2,12],[1.304...|
|    0|(20,[0,1,2,5,13,1...|
|    1|(20,[1,12,14,18],...|
|    1|(20,[1,10,12,13,1...|
|    1|(20,[2,8],[1.3049...|
|    1|(20,[2,3,4,5,6,7,...|
|    1|(20,[2,10],[2.609...|
|    1|(20,[8,10],[1.166...|
|    1|(20,[0,1,2,3,4,6,...|
|    0|(20,[3,4,13],[1.2...|
|    1|(20,[0,1,2,3,4,5,...|
|    0|(20,[1,3,4,11,13,...|
|    1|(20,[2,3,12],[1.3...|
|    1|(20,[5,13,19],[1....|
+-----+--------------------+
only showing top 20 rows

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(20,[0],[1.139699...|
|    0|(20,[0],[2.279398...|
|    0|(20,[0,1,2,3,4,5,...|
|    0|(20,[0,1,2,3,4,6,...|
|    0|(20,[0,1,2,3,4,8,...|
|    0|(20,[0,1,2,5,13,1...|
|    0|(20,[0,1,3

In [28]:

# Train and check
nb = NaiveBayes(modelType='multinomial')
model = nb.fit(train_data)
# model.save('model/nbSpamFilter.model')

predictions = model.transform(test_data)
predictions.select("label", "prediction", "features").show()

evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print ("Model Accuracy: ", accuracy)


+-----+----------+--------------------+
|label|prediction|            features|
+-----+----------+--------------------+
|    0|       0.0|(20,[0],[1.139699...|
|    0|       0.0|(20,[0],[2.279398...|
|    0|       1.0|(20,[0,1,2,3,4,5,...|
|    0|       1.0|(20,[0,1,2,3,4,6,...|
|    0|       1.0|(20,[0,1,2,3,4,8,...|
|    0|       1.0|(20,[0,1,2,5,13,1...|
|    0|       0.0|(20,[0,1,3,5,8,11...|
|    0|       0.0|(20,[0,1,3,10,11,...|
|    0|       0.0|(20,[0,1,3,11,14,...|
|    0|       0.0|(20,[0,1,3,12,14,...|
|    0|       0.0|(20,[0,1,3,13],[1...|
|    0|       0.0|(20,[0,1,5,9,13,1...|
|    0|       0.0|(20,[0,1,5,11,13,...|
|    0|       0.0|(20,[0,1,6,9,10,1...|
|    0|       1.0|(20,[0,1,7],[1.13...|
|    0|       0.0|(20,[0,1,7,8,19],...|
|    0|       0.0|(20,[0,1,9,10,13,...|
|    0|       0.0|(20,[0,1,9,19],[2...|
|    0|       0.0|(20,[0,2,7,10,11,...|
|    0|       0.0|(20,[0,2,9,14,18]...|
+-----+----------+--------------------+
only showing top 20 rows

Model Accuracy

# Testing

In [29]:
txt = "#Hey guys! 123 Please join me in my fight🐯🐯🐯 to help abused/mistreated animals! All  fund will go to helping pay for vet bills/and or helping them find homes! I  will place an extra emphasis on helping disabled animals, ones otherwise  would just be put to sleep by other animal organizations. Donate please. http://www.gofundme.com/Angels-n-Wingz﻿ ROAAAAARRRRRR ﻿"

def clear_out(text):
    mo = text.lower()
    mo = re.sub(r'https?:\/\/\S+', '', mo)               # Removing all links
    mo = re.sub(r"/", ' ', mo)
    mo = re.sub(r'@[A-Za-z0-9]+', '', mo)              # Removing all @mentions
    mo = re.sub(r'#', '', mo)                           
    mo = re.sub(r"([^0-9A-Za-z \t])", "", mo)
    mo = re.sub(r"\w*\d\w*", "", mo)
    mo = re.sub(r"^\d+\w|^\w+\d+\w|^\w+\d$", "", mo)
    mo = mo.encode('ascii', 'ignore').decode('ascii')
    
    return mo

print(clear_out(txt))

hey guys  please join me in my fight to help abused mistreated animals all  fund will go to helping pay for vet bills and or helping them find homes i  will place an extra emphasis on helping disabled animals ones otherwise  would just be put to sleep by other animal organizations donate please  roaaaaarrrrrr 


In [ ]:
def tfidf_vectorizer(text):
    
    data = [{'CONTENT': text}]
    docs = spark.createDataFrame(data)
    
    print(docs.show())
    
    tokenize = Tokenizer(inputCol='CONTENT', outputCol='WORDS')
    words_data = tokenize.transform(docs.dropna())

    hashingTF = HashingTF(inputCol="WORDS", outputCol="tf_features", numFeatures=20)
    featurized_data = hashingTF.transform(words_data)

    idf = IDF(inputCol="tf_features", outputCol="features")
    idfModel = idf.fit(featurized_data)
    rescaledData = idfModel.transform(featurized_data)

    # rescaledData.show()
    final_data = rescaledData.drop('CONTENT', 'WORDS', 'tf_features')
    print(final_data.first()['features'])
    
tfidf_vectorizer(txt)

+--------------------+
|             CONTENT|
+--------------------+
|#Hey guys! 123 Pl...|
+--------------------+

None
(20,[0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])


In [ ]:
from YTWebApp import DataPrep

dp = DataPrep.data_prepper()

print(dp.clean_data(text=txt))

(20,[0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])


In [37]:
def test():
    for i in range(10):
        yield (i, i+1)
        
def test2():
    
    for i in test():
        print(i[0], end=" ")

test2()

0 1 2 3 4 5 6 7 8 9 